# Lab 03: Embedded Deep Learning

In this lab session, we will optimize the deep learning model that was trained in the last session. Later we will put the quantized model to the ESP32 MCU.

Open in google colab -> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg#left)](https://colab.research.google.com/github/SuperChange001/deeplearning_labs/blob/main/Lab03/Lab03.ipynb)

## Setup

First, we import some libraries for image processing and utils, as well as TensorFlow. Note that the module `image_dataset_from_directory` is necessary for downloading our data set from Google.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

# Set the seed value for experiment reproducibility.
seed = 32
tf.random.set_seed(seed)
np.random.seed(seed)

## Import the Gesture dataset for evaluating the pretrianed model

Download and extract the `zip` file containing the datasets with `tf.keras.utils.get_file`. 

*Tips: change the code respectively if you have a model for other task.*

In [2]:
# Download our dataset used for training
TRAIN_SET_URL = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'
path_to_zip = tf.keras.utils.get_file('rps.zip', origin=TRAIN_SET_URL, extract=True, cache_dir='/content')
train_dir = os.path.join(os.path.dirname(path_to_zip), "rps")

# As well as the validation dataset
VAL_SET_URL = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'
path_to_zip2 = tf.keras.utils.get_file('rps-test-set.zip', origin=VAL_SET_URL, extract=True, cache_dir='/content')
validation_dir = os.path.join(os.path.dirname(path_to_zip2), "rps-test-set")

29532160/29516758 [==============================] - 2s 0us/step


Then we can generate tf.data.Dataset from image files in a directory.



In [3]:
BATCH_SIZE = 32
IMG_SIZE = (96, 96) 

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 2520 files belonging to 3 classes.
Found 372 files belonging to 3 classes.


Lets display some images of our dataset, as well as the class names.

### Split test set and validation set
We are now taking a fifth of the validation dataset to use as our test set. The validation set will be used for observing if we got overfitting during training while the test set is for the final test after training:

In [4]:
val_batches = tf.data.experimental.cardinality(validation_dataset)

test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 10
Number of test batches: 2


## Upload model and load the model from hard disk

Fetch the example model from github, you can also upload your own model to colab:

In [6]:
import io
import h5py
import requests

url = 'https://github.com/SuperChange001/deeplearning_labs/raw/main/Lab03/pretrained_models/model_rps.h5'
r = requests.get(url, allow_redirects=True)

with open('/content/model_rps.h5', 'wb') as f:
    f.write(r.content)

Alternative way for fetching the pretrained model:

In [8]:
!wget https://github.com/SuperChange001/deeplearning_labs/raw/main/Lab03/pretrained_models/model_rps.h5

--2022-08-15 14:20:28--  https://github.com/SuperChange001/deeplearning_labs/raw/main/Lab03/pretrained_models/model_rps.h5
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SuperChange001/deeplearning_labs/main/Lab03/pretrained_models/model_rps.h5 [following]
--2022-08-15 14:20:28--  https://raw.githubusercontent.com/SuperChange001/deeplearning_labs/main/Lab03/pretrained_models/model_rps.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3441936 (3.3M) [application/octet-stream]
Saving to: ‘model_rps.h5’

model_rps.h5        100%[===================>]   3.28M  --.-KB/s    in 0.01s   

2022-08-

Load model from hard disk

In [9]:
model = tf.keras.models.load_model('/content/model_rps.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 96, 96, 3)        0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 96, 96, 3)        0         
 a)                                                              
                                                                 
 mobilenetv2_0.35_96 (Functi  (None, 3, 3, 1280)       410208    
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                      

Test Accuracy of the loaded model

In [10]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

2/2 [==============================] - 12s 70ms/step - loss: 0.2492 - accuracy: 0.9219
Test accuracy : 0.921875


## Convert model

In [12]:
# Parameters setting
optimization_config = [tf.lite.Optimize.DEFAULT]

### Convert to a TensorFlow Lite model

In [13]:
#TF Lite model without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

### Convert using dynamic range quantization

In [14]:
#TF Lite model with dynamic range quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = optimization_config

tflite_model_dynamic_range = converter.convert()

### Convert using float fallback quantization

In [15]:
#Extracts sample images needed for float fallback and full integer quantization 
def representative_data_gen():
  for input in train_dataset.take(4):
    for input_value in tf.data.Dataset.from_tensor_slices(np.array(input[0])).batch(1).take(32):
      yield [input_value]

#TF Lite model with Float Fallback quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = optimization_config
converter.representative_dataset = representative_data_gen

tflite_model_float_fallback = converter.convert()

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


### Convert using integer-only quantization

In [16]:
#TF Lite model with Full integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = optimization_config
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to int8 
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


### Save the models as files

You'll need a `.tflite` file to deploy your model on other devices. So let's save the converted models to files and then load them when we run inferences below.

In [17]:
import pathlib

tflite_models_dir = pathlib.Path("/content/rps_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:

tflite_model_file = tflite_models_dir/"rps_model.tflite"
tflite_model_file.write_bytes(tflite_model)

# Save the dynamic range quantized model:

tf_model_dynamic_range_file = tflite_models_dir/"rps_model_dynamic_range.tflite"
tf_model_dynamic_range_file.write_bytes(tflite_model_dynamic_range)

# Save the float fallback quantized model:

tflite_model_float_fallback_file = tflite_models_dir/"rps_model_float_fallback.tflite"
tflite_model_float_fallback_file.write_bytes(tflite_model_float_fallback)

# Save the integer only quantized model:

tflite_model_quant_file = tflite_models_dir/"rps_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

627696

### Test the models

In [19]:
def evaluate_model(tflite_file, dataset, model_type):
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))

  interpreter.allocate_tensors()
  
  input_details = interpreter.get_input_details()[0]
  output_details =  interpreter.get_output_details()[0]

  total_seen = 0
  num_correct = 0
  is_int8_quantized = (input_details['dtype'] == np.int8)
  
  for image_batch, labels_batch in dataset:
    for i in range(tf.shape(image_batch)[0]):
      test_image = image_batch[i]
      

      if is_int8_quantized:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

      test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
      interpreter.set_tensor(input_details["index"], test_image)
      interpreter.invoke()
      output = interpreter.get_tensor(output_details["index"])[0]

      output = np.argmax(output)

      if labels_batch[i] == output:
        num_correct += 1
      total_seen += 1

      if total_seen % 50 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))
  print('Num images: {0:}, Accuracy: {1:.4f}, Type: {2:}'.format(total_seen, float(num_correct / total_seen), model_type))

In [20]:
#Check accuracy on the test subset
evaluate_model(tflite_model_file, test_dataset, model_type="Float")
evaluate_model(tf_model_dynamic_range_file, test_dataset, model_type="Dynamic Range")
evaluate_model(tflite_model_float_fallback_file, test_dataset, model_type="Float Fallback")
evaluate_model(tflite_model_quant_file, test_dataset, model_type="Integer Quantized")
model.evaluate(test_dataset)

Accuracy after 50 images: 0.800000
Num images: 64, Accuracy: 0.7969, Type: Float
Accuracy after 50 images: 0.920000
Num images: 64, Accuracy: 0.9219, Type: Dynamic Range
Accuracy after 50 images: 0.960000
Num images: 64, Accuracy: 0.9688, Type: Float Fallback
Accuracy after 50 images: 0.960000
Num images: 64, Accuracy: 0.9531, Type: Integer Quantized
2/2 [==============================] - 1s 67ms/step - loss: 0.2270 - accuracy: 0.9375


[0.2269519865512848, 0.9375]

In [22]:
#Check accuracy on all validation data
evaluate_model(tflite_model_file, validation_dataset, model_type="Float")
evaluate_model(tf_model_dynamic_range_file, validation_dataset, model_type="Dynamic Range")
evaluate_model(tflite_model_float_fallback_file, validation_dataset, model_type="Float Fallback")
evaluate_model(tflite_model_quant_file, validation_dataset, model_type="Integer Quantized")
model.evaluate(validation_dataset)

Accuracy after 50 images: 0.960000
Accuracy after 100 images: 0.970000
Accuracy after 150 images: 0.940000
Accuracy after 200 images: 0.940000
Accuracy after 250 images: 0.924000
Accuracy after 300 images: 0.916667
Num images: 308, Accuracy: 0.9188, Type: Float
Accuracy after 50 images: 0.920000
Accuracy after 100 images: 0.920000
Accuracy after 150 images: 0.900000
Accuracy after 200 images: 0.905000
Accuracy after 250 images: 0.916000
Accuracy after 300 images: 0.923333
Num images: 308, Accuracy: 0.9253, Type: Dynamic Range
Accuracy after 50 images: 0.960000
Accuracy after 100 images: 0.960000
Accuracy after 150 images: 0.960000
Accuracy after 200 images: 0.970000
Accuracy after 250 images: 0.972000
Accuracy after 300 images: 0.963333
Num images: 308, Accuracy: 0.9643, Type: Float Fallback
Accuracy after 50 images: 0.940000
Accuracy after 100 images: 0.960000
Accuracy after 150 images: 0.966667
Accuracy after 200 images: 0.965000
Accuracy after 250 images: 0.964000
Accuracy after 300

[0.30543866753578186, 0.9155844449996948]

In [23]:
print("Float model in KB:", os.path.getsize(tflite_model_file) / float(2**10))
print("Dynamic Range model in KB:", os.path.getsize(tf_model_dynamic_range_file) / float(2**10))
print("Float fallback model in KB:", os.path.getsize(tflite_model_float_fallback_file) / float(2**10))
print("Integer Quantized model in KB:", os.path.getsize(tflite_model_quant_file) / float(2**10))

Float model in KB: 1573.6015625
Dynamic Range model in KB: 592.25
Float fallback model in KB: 613.3046875
Integer Quantized model in KB: 612.984375


So you now have an integer quantized a model with almost no difference in the accuracy, compared to the float model.

To learn more about other quantization strategies, read about [TensorFlow Lite model optimization](https://www.tensorflow.org/lite/performance/model_optimization).

## Generate a TensorFlow Lite for MicroControllers Model
Convert the TensorFlow Lite model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.

In [18]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
MODEL_TFLITE = "/content/rps_tflite_models/rps_model_quant.tflite"
MODEL_TFLITE_MICRO = "/content/rps_tflite_models/rps_model_quant.cc"
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/rps_model_tflite/g' {MODEL_TFLITE_MICRO}

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

### Deploy to a Microcontroller
We now need to move to PC side.

In [24]:
from google.colab import files
files.download('/content/rps_tflite_models/rps_model_quant.cc') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>